In [ ]:
from google.colab import drive
drive.mount('/content/drive')


2) Clone repo

In [ ]:
%cd '/content/drive/MyDrive/AI/AI_practise/gitclone'
!ls 
#! git clone https://github.com/hoaquocphan/face_recognition
%cd 'face_recognition'
!pwd


3) Prepare data

In [ ]:
!ls
#! pip install onnxruntime
#!python3 prepare_data.py --d UTKFace
#!rm -r model


In [ ]:
#!python3 train_model.py 


4) Run script train model train_model.py or run command !python3 train_model.py 

4-1) Load library

In [ ]:
from os import walk
import os
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import cv2

def preprocess_image(img):
    image = cv2.imread(img)
    input_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    input_img = cv2.resize(input_img, (182, 182)) 
    return input_img

def get_label(dir,num_class):
    image_name = []
    labels = []
    count = 0
    for (dirpath, dirnames, filenames) in walk(dir):
        image_name.extend(filenames)
    for image in image_name:
        if image.split("_")[0] not in labels:
            count +=1
        if count > num_class:
            break
        labels.append(image.split("_")[0])

    labels = [int(numeric_string) for numeric_string in labels]
    labels=np.array(labels)
    return labels

def get_images(dir,num_class):
    image_name=[]
    images =[]
    image_name2=[]
    count = 0
    for (dirpath, dirnames, filenames) in walk(dir):
        image_name.extend(filenames)
    for image in image_name:
        if image.split("_")[0] not in image_name2:
            count +=1
        if count > num_class:
            break
        image_name2.append(image.split("_")[0])
        image_file = dir + "/" + image
        image=preprocess_image(image_file)
        images.append(image)
    images = np.array(images)
    images = images/255.0
    return images


def create_model(num_class):
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(182, 182)),
        tf.keras.layers.Dense(640, activation='relu'),
        tf.keras.layers.Dense(int(num_class)) #number of class
    ])
    model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    model.summary()
    return model



4-2) Processing and getting data

In [ ]:
from os import walk
import os
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import cv2
from lib import get_label, get_images, create_model
import argparse
print(tf.__version__)


num_class=48


print("loading train label")
if os.path.isfile("train_labels.npy"):
    print("load file numpy")
    train_labels = np.load('train_labels.npy') 
else:
    train_labels = get_label("train",num_class)
    np.save('train_labels.npy', train_labels)

print("loading validation label")
if os.path.isfile("validation_labels.npy"):
    print("load file numpy")
    validation_labels = np.load('validation_labels.npy')
else:
    validation_labels = get_label("validation",num_class)
    np.save('validation_labels.npy', validation_labels)

print("loading train image")
if os.path.isfile("train_images.npy"):
    print("load file numpy")
    train_images = np.load('train_images.npy')
else:
    train_images = get_images("train",num_class)
    np.save('train_images.npy', train_images)

print("loading validation image")
if os.path.isfile("validation_images.npy"):
    print("load file numpy")
    validation_images = np.load('validation_images.npy')
else:
    validation_images = get_images("validation",num_class)
    np.save('validation_images.npy', validation_images)


In [ ]:
print(train_labels.shape)
print(validation_labels.shape)
print(train_images.shape)
print(validation_images.shape)
!ls
#rm train_images.npy train_labels.npy

4-3) Train model

In [ ]:
model=create_model(num_class)


checkpoint_path = "model/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,save_weights_only=True,verbose=1)
# Train model by train image 
print("training model")
model.fit(train_images, train_labels, epochs=50,validation_data=(validation_images, validation_labels), callbacks=[cp_callback])



test_loss, test_acc = model.evaluate(validation_images,  validation_labels, verbose=2)
print("model, accuracy: {:5.2f}%".format(100 * test_acc))